<a href="https://colab.research.google.com/github/marianaso/PySpark/blob/main/AnaliseEnem2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 15.4 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824028 sha256=02d6498e8b5a5f432bcb061dcf867fae38b67dddd9300dea8af30e1e1e5279d2
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .appName("Enem 2020") \
    .config('spark.ui.port', '4050') \
    .getOrCreate()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile

In [ ]:
zipfile.ZipFile('/content/drive/MyDrive/Spark/microdados_enem_2020.zip', 'r', ).extractall('/content/drive/MyDrive/Spark/ENEM/')

In [ ]:
dados = spark.read.csv('/content/drive/MyDrive/Spark/ENEM/DADOS/MICRODADOS_ENEM_2020.csv', sep=';', inferSchema=True, header=True, encoding = 'utf-8')

In [ ]:
from pyspark.sql import functions as f

In [ ]:
dados.createOrReplaceTempView("dados")

In [ ]:
##Quantos alunos não quiseram declarar a cor/raça em 2020 (Entenda a opção “não declarado” nessa pergunta)?

spark\
    .sql("""
        SELECT count(*) 
            FROM dados
            WHERE TP_COR_RACA = 0
    """)\
    .show()

In [ ]:
##Qual é o número de alunos do Sexo Feminino que estudaram em escola no estado de São Paulo?

spark\
    .sql("""
        SELECT count(*) 
            FROM dados
            WHERE TP_SEXO = 'F' AND SG_UF_ESC = 'SP'
    """)\
    .show()

+--------+
|count(1)|
+--------+
|   91480|
+--------+



In [ ]:
##Quantos alunos do sexo feminino que estudaram em escola no estado do Rio Grande do Sul possuem EXATAMENTE uma geladeira em casa?

spark\
    .sql("""
        SELECT count(*) 
            FROM dados
            WHERE TP_SEXO = 'F' AND SG_UF_ESC = 'RS' AND Q012 = 'B'
    """)\
    .show()

+--------+
|count(1)|
+--------+
|   18680|
+--------+



In [ ]:
##Qual é o segundo estado brasileiro (considere a coluna SG_UF_ESC) em que estudaram mais alunos no ENEM 2020?

spark\
    .sql("""
        SELECT SG_UF_ESC, COUNT(SG_UF_ESC) AS NUMERO_ALUNOS
            FROM dados
            GROUP BY SG_UF_ESC
            ORDER BY NUMERO_ALUNOS DESC
    """)\
    .show()

In [ ]:
##Qual é o estado brasileiro (considere a coluna SG_UF_ESC) possui o menor número de alunos cuja mãe possui ensino superior completo?

spark\
    .sql("""
        SELECT SG_UF_ESC, COUNT(SG_UF_ESC) AS NUMERO_ALUNOS
            FROM dados
            WHERE Q002 = 'F' OR Q002 = 'G'
            GROUP BY SG_UF_ESC
            ORDER BY NUMERO_ALUNOS ASC
    """)\
    .show()

In [ ]:
##Qual é o segundo estado brasileiro (considere a coluna SG_UF_ESC) que possui o maior número de pessoas na faixa “entre 26 e 30 anos”?
spark\
    .sql("""
        SELECT SG_UF_ESC, COUNT(SG_UF_ESC) AS NUMERO_ALUNOS
            FROM dados
            WHERE TP_FAIXA_ETARIA = 11
            GROUP BY SG_UF_ESC
            ORDER BY NUMERO_ALUNOS DESC
    """)\
    .show()

In [ ]:
##Qual é o estado brasileiro (considere a coluna SG_UF_ESC) que possui o TERCEIRO maior número de alunos cuja residência possui PELO MENOS 2 banheiros?

spark\
    .sql("""
        SELECT SG_UF_ESC, COUNT(SG_UF_ESC) AS NUMERO_ALUNOS
            FROM dados
            WHERE Q008 IN ('C', 'D', 'E')
            GROUP BY SG_UF_ESC
            ORDER BY NUMERO_ALUNOS DESC
    """)\
    .show()

In [ ]:
##Quantos alunos do sexo feminino se autodeclararam pretos?

spark\
    .sql("""
        SELECT count(*) 
            FROM dados
            WHERE TP_COR_RACA = 2 AND TP_SEXO = 'F'
    """)\
    .show()

+--------+
|count(1)|
+--------+
|  444346|
+--------+



In [ ]:
##Quantos alunos estrangeiros fizeram o ENEM 2020?

spark\
    .sql("""
        SELECT count(*) 
            FROM dados
            WHERE TP_NACIONALIDADE = 3
    """)\
    .show()

+--------+
|count(1)|
+--------+
|    8036|
+--------+



In [ ]:
##Qual é a diferença da nota média em matemática dos alunos que estudaram o ensino médio em escola pública e em escola privada?

spark\
    .sql("""
        SELECT TP_ESCOLA, MEAN(NU_NOTA_MT) AS NOTA_MEDIA_MAT
            FROM dados
            GROUP BY TP_ESCOLA
    """)\
    .show()

In [ ]:
##Qual é o estado brasileiro (considere a coluna SG_UF_ESC) que possui o maior número de alunos do sexo feminino indígenas?

spark\
    .sql("""
        SELECT SG_UF_ESC, COUNT(SG_UF_ESC) AS NUMERO_ALUNOS
            FROM dados
            WHERE TP_COR_RACA = 5 AND TP_SEXO = 'F'
            GROUP BY SG_UF_ESC
            ORDER BY NUMERO_ALUNOS DESC
    """)\
    .show()

In [ ]:
##Qual é a diferença entre o número de alunos cujo pai possui pós-graduação completa e o número de alunos cuja mãe possui pós-graduação completa?

##Pai com pos-graducao completa
spark\
    .sql("""
        SELECT COUNT(*) AS NUMERO_ALUNOS
            FROM dados
            WHERE Q001 = 'G'
    """)\
    .show()

+-------------+
|NUMERO_ALUNOS|
+-------------+
|       239720|
+-------------+



In [ ]:
spark\
    .sql("""
        SELECT COUNT(*) AS NUMERO_ALUNOS
            FROM dados
            WHERE Q002 = 'G'
    """)\
    .show()

In [ ]:
##Quantos alunos cuja residência possui ATÉ 2 carros estudaram na região NORDESTE do Brasil?

spark\
    .sql("""
        SELECT COUNT(*) AS NUMERO_ALUNOS
            FROM dados
            WHERE Q010 IN ('B', 'C') AND SG_UF_ESC IN ('MA','PI','CE','RN','PB','PE','AL','SE','BA')
    """)\
    .show()

In [ ]:
##Quantos alunos que estudaram em escolas em zona rural possuem internet em casa?

spark\
    .sql("""
        SELECT COUNT(*) AS NUMERO_ALUNOS
            FROM dados
            WHERE TP_LOCALIZACAO_ESC = 2 AND Q025 = 'B'
    """)\
    .show()

In [ ]:
##Qual é o nome do município que contém a SEGUNDA maior quantidade total de inscritos no ENEM 2020?

spark\
    .sql("""
        SELECT NO_MUNICIPIO_ESC, COUNT(NO_MUNICIPIO_ESC) AS NUMERO_ALUNOS
            FROM dados
            GROUP BY NO_MUNICIPIO_ESC
            ORDER BY NUMERO_ALUNOS DESC
    """)\
    .show()

+----------------+-------------+
|NO_MUNICIPIO_ESC|NUMERO_ALUNOS|
+----------------+-------------+
|       S�o Paulo|        38277|
|       Fortaleza|        27783|
|  Rio de Janeiro|        26431|
|        Bras�lia|        19840|
|          Manaus|        17120|
|  Belo Horizonte|        11826|
|          Recife|        10735|
|        Salvador|        10500|
|        Curitiba|         9868|
|           Bel�m|         9137|
|         Goi�nia|         8920|
|        S�o Lu�s|         6573|
|        Teresina|         5776|
|    Porto Alegre|         5650|
|          Macei�|         5070|
|     Jo�o Pessoa|         5005|
|    Campo Grande|         4767|
|       Guarulhos|         4689|
|           Natal|         4519|
|        Campinas|         4323|
+----------------+-------------+
only showing top 20 rows



In [ ]:
dados.printSchema()

root
 |-- NU_INSCRICAO: long (nullable = true)
 |-- NU_ANO: integer (nullable = true)
 |-- TP_FAIXA_ETARIA: integer (nullable = true)
 |-- TP_SEXO: string (nullable = true)
 |-- TP_ESTADO_CIVIL: integer (nullable = true)
 |-- TP_COR_RACA: integer (nullable = true)
 |-- TP_NACIONALIDADE: integer (nullable = true)
 |-- TP_ST_CONCLUSAO: integer (nullable = true)
 |-- TP_ANO_CONCLUIU: integer (nullable = true)
 |-- TP_ESCOLA: integer (nullable = true)
 |-- TP_ENSINO: integer (nullable = true)
 |-- IN_TREINEIRO: integer (nullable = true)
 |-- CO_MUNICIPIO_ESC: integer (nullable = true)
 |-- NO_MUNICIPIO_ESC: string (nullable = true)
 |-- CO_UF_ESC: integer (nullable = true)
 |-- SG_UF_ESC: string (nullable = true)
 |-- TP_DEPENDENCIA_ADM_ESC: integer (nullable = true)
 |-- TP_LOCALIZACAO_ESC: integer (nullable = true)
 |-- TP_SIT_FUNC_ESC: integer (nullable = true)
 |-- CO_MUNICIPIO_PROVA: integer (nullable = true)
 |-- NO_MUNICIPIO_PROVA: string (nullable = true)
 |-- CO_UF_PROVA: integer 